In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scanpy as sc
import os
import cv2

from micron2.codexutils import ImageMontage, layout_cells

from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
def read_color_config(fname):
    df = pd.read_csv(fname, index_col=None, header=None)
    df.columns = ['channel', 'color', 'low', 'high']
    colors = {}
    saturations = {}
    for i in range(df.shape[0]):
        colors[df.loc[i,'channel']] = df.loc[i,'color']
        saturations[df.loc[i,'channel']] = (df.loc[i,'low'], df.loc[i,'high'])
    return colors, saturations


In [ ]:
# channel_groups = [
#     ['DAPI', 'CD45', 'PanCytoK', 'Ki-67',],
#     ['CD20', 'CD3e', 'CD8', 'CD4',],
#     ['CD68', 'C1q', 'CD31', 'CD11c',],
#     ['IL7R', 'CD103', 'CD134', 'CD69',],
#     ['CXCL13', 'CXCR5', 'PD-1', 'aSMA',],
#     ['OX40L', 'HLA-DR', 'LAG3', 'IgA',],
#     ['CD89', 'PNaD', 'PD-L1', 'CD80',],
#     ['TIM3', 'CD45RO', 'CD40LG', 'FOXP3',],
#     ['CD64', 'GZMB', 'CD40', 'CD45RA',],
#     ['CD138', 'IgM', 'PDGFRb']
# ]

channel_groups = [
#     ['DAPI', 'CD45', 'PanCytoK', 'CD68',], # Epithelial
#     ['DAPI', 'CD4', 'CD3e', 'CD8', 'CD20'], # Tcell
#     ['DAPI', 'CD3e', 'CD68', 'PanCytoK', 'CD45'], # Mac
#     ['DAPI', 'PD-1', 'CD8', 'LAG3', 'CXCL13'], # Trm
#     ['DAPI', 'PD-1', 'CD4', 'CXCL13'], # Tfh
    ['DAPI', 'aSMA', 'PDGFRb', 'CD31', 'PanCytoK'] # Stromal
#     ['DAPI', 'CXCR5', 'CD20', 'CD3e', 'CD68'], # Bcell
]

In [ ]:
all_samples = !ls -d /storage/codex/preprocessed_data/*Bladder*
all_samples = [os.path.basename(s) for s in all_samples]
all_samples = [s for s in all_samples if 'reg0' not in s]
all_samples

In [ ]:
sources

In [ ]:
def filter_coordinates(coords, s):
    valid_coords = np.ones(coords.shape[0], dtype=np.bool)
    max_1, max_2 = np.max(coords, axis=0)
    i = coords[:,0]<(s+1)
    i += coords[:,1]<(s+1)
    i += coords[:,0]>(max_1-s)
    i += coords[:,1]>(max_2-s)
    valid_coords[i] = 0
    return valid_coords

In [ ]:
celltype = 'Bcell'
celltype_col = 'celltype_gating'
data_home = '/storage/codex/preprocessed_data'
outdir = f'/storage/tmp-outgoing/codex-cell-images/bladder/{celltype}'
s = 64
ncol = 8
ncells = 64

colors, saturations = read_color_config('/home/ingn/tmp/micron2-data/bladder/color_config.csv')
channel_groups = [
#     ['DAPI', 'CD4', 'CD3e', 'CD8', 'CD20'], # Tcell
#     ['DAPI', 'PanCytoK', 'KRT13', 'KRT17', 'CDH12', 'CDH18']
#     ['DAPI', 'KRT20', 'CK5-6', 'CDH12', 'CDH18']
    ['DAPI', 'CD20', 'CD3e', 'CD68'], # Bcell
    
]

for sample in all_samples:
    adataf = f'{data_home}/{sample}/{sample}.h5ad'
    outf = f'{outdir}/{sample}.png'
    
    adata = sc.read_h5ad(adataf)
    channels = adata.uns['channels']
    coords = adata.obsm['coordinates'].copy()
    coords[:,1] = -coords[:,1]
    
    valid_coords = filter_coordinates(coords, s)
    
    import glob
    def get_source(image_home, ch):
        lst = glob.glob(f'{image_home}/*.tif')
        # srch = f'_{ch}_' if ch != 'DAPI' else f'_{ch}-011'
        srch = f'_{ch}_' if ch != 'DAPI' else f'_{ch}1_'
        for l in lst:
            if srch in l:
                return l

    sources = {ch: get_source(f'{data_home}/{sample}/images', ch) for ch in channels}
    montager = ImageMontage(sources=sources, colors=colors, saturations=saturations,
                            channel_groups=channel_groups)

    indices = np.argwhere((adata.obs[celltype_col].values == celltype)&(valid_coords)).flatten()
    if len(indices) < ncells:
        continue
        
    idx = np.random.choice(indices, ncells, replace=False)
    bboxes = []
    for i in idx:
        bbox = [coords[i,1]-s, coords[i,1]+s, coords[i,0]-s, coords[i,0]+s]
        bboxes.append(bbox)

    m = montager.montage_several(bboxes)


    layer = 0
    out = layout_cells(m, layer, ncol=ncol)
    out_legend = montager.add_color_legend(out, layer)

    print(f'{out_legend.shape} --> {outf}')
    cv2.imwrite(outf, out_legend[:,:,::-1])
    